In [2]:
import dotenv 
import os 

dotenv.load_dotenv()

True

In [4]:
from pymongo import MongoClient

host = "localhost"
port = 27017
client = MongoClient(f"mongodb://{host}:{port}/")
db = client["LLMQueryAgent"]
print(db.list_collection_names())
collection = db["Functional"]

['Functional']


In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an expert AI assistant specializing in generating efficient and accurate MongoDB queries.""" + \
                   """A database entry is as follows: {schema}.\nHere is an example of mongo query interaction you must follow this assisstant response: {one_shot}.\n""" + \
                   """When returning a query, output only the query itself with no extra words or explanations or punctuations,""" + \
                   """so that an user can directly run the command."""),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{query}")
    ]
)

prompt.invoke(
    {
        "schema": collection.find_one({}),
        "one_shot": "dummy",
        "history": [
            {"role": "user", "content": ""},
            {"role": "assistant", "content": ""}
        ],
        "query": "What is the capital of France?"
    }
)


ChatPromptValue(messages=[SystemMessage(content="You are an expert AI assistant specializing in generating efficient and accurate MongoDB queries.A database entry is as follows: {'_id': ObjectId('67ced2ca97c2a94c1b06d3d1'), 'frame_time': datetime.datetime(2024, 11, 4, 12, 5, 5, 428000), 'frame_time_epoch': 1730721905428675000, 'frame_number': '1496', 'protocol': 'HTTP2/JSON', 'src': 'UDR', 'dst': 'UDM', 'tcp_srcport': '2760', 'tcp_dstport': '34332', 'udp_srcport': '', 'udp_dstport': '', 'sctp_srcport': '', 'sctp_dstport': '', 'info': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404', 'Error_Markers': {'type': 'http2', 'status': '404', 'request': '/nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations', 'method': 'get'}, 'Message_Identifier': {'message': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404'}, 'job_id': '98ad489a-9158-44f8-8f04-7b7e362e0d74'}.\nH

In [6]:
from langchain_openai import ChatOpenAI

# docker run -it -p 8000:8000  tirthankar95/qwen7b

# llama-server -m qwen2.5-7b-instruct-q4_0.gguf \
#     --threads -1 --host 0.0.0.0 --port 8000 \
#     --log-file chat_llm \
#     --ctx-size 8192 \
#     --n-predict 512 \
#     --temp 0.5 \
#     --top-k 10 \
#     --top-p 0.9 \
#     --min-p 0.1 \
#     --repeat-penalty 1.1 \
#     --mlock \
#     --batch-size 16

openai_api_key = "NA"
openai_api_base = "http://localhost:8000/v1"
model_name = "qwen2.5-7b-instruct-q4_0.gguf"
model = ChatOpenAI(
    api_key = openai_api_key,
    base_url = openai_api_base,
    model_name = model_name
)
model_gpt = ChatOpenAI(
    api_key = os.environ["OPENAI_API_KEY"],
    model_name = "gpt-3.5-turbo"
)

In [7]:
import re 

def mongo_exec(ai_message):
    try:
        print(f'AI mongo query: {ai_message}')
        return f"The result of the query is: {eval(ai_message)}."
    except Exception as e:
        pattern = re.compile(r"\bcollection")
        if pattern.search(ai_message):
            return f"Extract only the query from {ai_message}."
        else:
            return f"Invalid query. Please try again."

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import hf_hub_download, list_repo_files
import os 

# Get model and tokenizer.
local_dir = "./embed_model/"
repo_id = "thenlper/gte-base"
filenames = list_repo_files(repo_id)
for file in filenames:
    if not os.path.exists(os.path.join(local_dir, file)):
        hf_hub_download(repo_id, file, local_dir=local_dir)

# Load embedding model.
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(model_name = repo_id, \
                           model_kwargs = model_kwargs, \
                           encode_kwargs = encode_kwargs)

In [10]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4

# Save one shot example in mongo-db
# DB_NAME = "LLM_MONGO"
# COLLECTION_NAME = "ONE_SHOT_EXAMPLES"
# ATLAS_VECTOR_SEARCH_INDEX_NAME = "LLM_MONGO_INDEX" 
# one_shot_collection = client[DB_NAME][COLLECTION_NAME]
# # Chat 1
# chat_example_1 = "user: Count the number of errors with source as UDR ?\n" + \
#                  """ai: collection.count_documents({'src': 'AMF'})"""
# chat1 = {
#     "_id": 1,
#     "one_shot": chat_example_1
# }
# one_shot_collection.insert_one(chat1)

# Chroma DB 
DB_NAME = "LLM_MONGO_1"
COLLECTION_NAME = "ONE_SHOT_EXAMPLES"
vector_store = Chroma(collection_name = COLLECTION_NAME, \
                      embedding_function = hf, \
                      persist_directory = f"./{DB_NAME}")
"""
Make all examples to be UDR, so that similarity search is better.
Otherwise, similarity search may return a one-shot if only the node name matches.
"""
examples = [
    Document(
        page_content = """user: Count the number of errors with source as UDR?\n assistant:collection.count_documents({"src": "UDR"})""",
        metadata = {"source": "manual_tmittra"}),
    Document(
        page_content = """user: Display errors with destination as UDR?\nassistant: list(collection.find({"dst": "UDR"}))""",
        metadata = {"source": "manual_tmittra"})
    ]
uuids = [str(uuid4()) for _ in range(len(examples))]
vector_store.add_documents(documents = examples, ids = uuids)
retriver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print(retriver.invoke("Count the number of errors with source as AMF?"))

[Document(id='d0439792-3466-4eb2-bbdb-09cab9a09867', metadata={'source': 'manual_tmittra'}, page_content='user: Count the number of errors with source as UDR?\n assistant:collection.count_documents({"src": "UDR"})')]


In [11]:
def vector_retriver(input_2llm):
    query, history = input_2llm['query'], input_2llm['history']
    rdocs = retriver.invoke(query)
    fmt_docx = "\n".join([doc.page_content for doc in rdocs])
    return {"schema": collection.find_one({}), \
            "one_shot": fmt_docx, \
            "history": history, \
            "query": query}

dummy_history = [{'role': 'user', 'content': 'Count the number of errors with destination as GNB?'}, \
                 {'role': 'assistant', 'content': 'The result of the query is: 2.'}]
vector_retriver({"query": "Count the number of errors with source as AMF?", \
                 "history": dummy_history})

{'schema': {'_id': ObjectId('67ced2ca97c2a94c1b06d3d1'),
  'frame_time': datetime.datetime(2024, 11, 4, 12, 5, 5, 428000),
  'frame_time_epoch': 1730721905428675000,
  'frame_number': '1496',
  'protocol': 'HTTP2/JSON',
  'src': 'UDR',
  'dst': 'UDM',
  'tcp_srcport': '2760',
  'tcp_dstport': '34332',
  'udp_srcport': '',
  'udp_dstport': '',
  'sctp_srcport': '',
  'sctp_dstport': '',
  'info': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404',
  'Error_Markers': {'type': 'http2',
   'status': '404',
   'request': '/nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations',
   'method': 'get'},
  'Message_Identifier': {'message': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404'},
  'job_id': '98ad489a-9158-44f8-8f04-7b7e362e0d74'},
 'one_shot': 'user: Count the number of errors with source as UDR?\n assistant:collection.count_documents({"src": "UDR"})',
 'hi

#### GPT-3.5-Turbo

In [15]:
from langchain_core.output_parsers import StrOutputParser
# Option 2: For extracting the number of errors.
# pipeline = [
#     {
#         "$match": { "src": "AMF" } 
#     },
#     {
#         "$count": "num_errors"  # $count requires a string field name
#     }
# ]

# result = list(collection.aggregate(pipeline))
# print(result) 
prompt_result = ChatPromptTemplate.from_messages(
    [
        ("system", "The result of the user query was: {result_query}")
    ]
)
chain = vector_retriver | prompt | model_gpt | StrOutputParser() | mongo_exec 
chain.invoke("Count the number of errors with destination as GNB?")

AI mongo query: collection.count_documents({"dst": "GNB"})


'The result of the query is:\n2.'

#### Qwen2.5-7b-Instruct-Q4

In [14]:
from langchain_core.output_parsers import StrOutputParser

MIN_CHAT_HISTORY = 3
def query_chain(query, history):
    print(history)
    history_sz = min(MIN_CHAT_HISTORY, len(history))
    try:
        chain_fn = vector_retriver | prompt | model_gpt | StrOutputParser() | mongo_exec 
        history_to_take = history[-history_sz:]
        history_to_take = [{'role': hist['role'], 'content': hist['content']} for hist in history_to_take]
        return chain_fn.invoke({"query": query, "history": history_to_take})
    except Exception as e:
        return str(e)

# Test 
query_chain("Count the number of errors with destination as GNB?", "")


AI mongo query: collection.count_documents({"dst": "GNB"})


'The result of the query is: 2.'

In [15]:
import gradio as gr 

gr.ChatInterface(query_chain, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[]
AI mongo query: list(collection.find({"dst": "UDR"}))
[{'role': 'user', 'metadata': None, 'content': 'from langchain_core.output_parsers import StrOutputParser', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'The result of the query is: [].', 'options': None}]
AI mongo query: list(collection.find({"dst": "UDR"}))
[{'role': 'user', 'metadata': None, 'content': 'from langchain_core.output_parsers import StrOutputParser', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'The result of the query is: [].', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'from langchain_core.output_parsers import StrOutputParser', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'The result of the query is: [].', 'options': None}]
AI mongo query: collection.count_documents({"dst": "GNB"})


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
local_dir = "./embed_model/"
repo_id = "thenlper/gte-base"

# Load embedding model.
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(model_name = repo_id, \
                           model_kwargs = model_kwargs, \
                           encode_kwargs = encode_kwargs)

In [9]:
from langchain_chroma import Chroma
COLLECTION_NAME = "ONE_SHOT_EXAMPLES"
DB_NAME = "LLM_MONGO_1"
vector_store = Chroma(collection_name = COLLECTION_NAME, \
                      embedding_function = hf, \
                      persist_directory = f"./{DB_NAME}")

retriver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
rdocs = retriver.invoke("Count the number of errors with destination as GNB?")
fmt_docx = "\n".join([doc.page_content for doc in rdocs])
print(retriver.invoke("Count the number of errors with source as AMF?"))

[Document(id='d0439792-3466-4eb2-bbdb-09cab9a09867', metadata={'source': 'manual_tmittra'}, page_content='user: Count the number of errors with source as UDR?\n assistant:collection.count_documents({"src": "UDR"})')]
